In [4]:
import pandas as pd
import numpy as np
import spacy
from dont_patronize_me import DontPatronizeMe
nlp = spacy.load(
    "en_core_web_lg"
)

In [5]:
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedShuffleSplit

In [6]:
d = DontPatronizeMe("data", "data")
d.load_task1()
df = d.train_task1_df
df.head()

,ids,text,labels
0,@@23953477,the ones in need of constant medical care are ...,0
1,@@4703096,nbc and spanish-language univision both declin...,0
2,@@25567226,a second t-home project is being launched in t...,0
3,@@1824078,camfed would like to see this trend reversed ....,1
4,@@1921089,kagunga village was reported to lack necessary...,0


In [7]:
from gensim.models import KeyedVectors
# Load vectors directly from the file
model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

In [8]:
def vectorize(text):
    words = []
    doc = nlp(text)
    for token in doc:
        if not token.is_stop:
            words.append(token.text) 
    vector_list = [model[word] for word in words if word in model.key_to_index]
    return np.average( np.array(vector_list), axis = 0)   

In [9]:
df["vec"] = df["text"].apply(vectorize)

C:\Users\misla\anaconda3\lib\site-packages\numpy\lib\function_base.py:380: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)


In [10]:
df.head()

,ids,text,labels,vec
0,@@23953477,the ones in need of constant medical care are ...,0,"[0.03873578, 0.082598515, -0.010548311, 0.1276..."
1,@@4703096,nbc and spanish-language univision both declin...,0,"[-0.063673906, -0.04059652, 0.026416015, -0.02..."
2,@@25567226,a second t-home project is being launched in t...,0,"[0.033587288, -0.02133493, -0.03506066, 0.0465..."
3,@@1824078,camfed would like to see this trend reversed ....,1,"[-0.01903358, 0.031904954, -0.014008742, 0.075..."
4,@@1921089,kagunga village was reported to lack necessary...,0,"[-0.039190117, 0.10054848, 0.052714955, -0.057..."


In [11]:
print(len(df))
df = df.dropna()
print(len(df))

10636
10635


In [121]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=111)

for train_index, test_index in sss.split(df['vec'], df['labels']):
    X_train, X_test = df.vec.iloc[train_index], df.vec.iloc[test_index]
    y_train, y_test = df.labels.iloc[train_index], df.labels.iloc[test_index]
    
    X_train = np.stack([a.reshape(300,) for a in X_train.values], axis = 0)
    X_test = np.stack([a.reshape(300,) for a in X_test.values], axis = 0)
    
    SVM = svm.SVC(C=10.0, kernel='poly',gamma='scale')
    SVM.fit(X_train, y_train)

    predictions_SVM = SVM.predict(X_test)
    print(classification_report(y_test, predictions_SVM))

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1927
           1       0.50      0.30      0.38       200

    accuracy                           0.91      2127
   macro avg       0.72      0.63      0.66      2127
weighted avg       0.89      0.91      0.90      2127

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1927
           1       0.44      0.26      0.32       200

    accuracy                           0.90      2127
   macro avg       0.68      0.61      0.63      2127
weighted avg       0.88      0.90      0.89      2127

              precision    recall  f1-score   support

           0       0.93      0.97      0.95      1927
           1       0.46      0.27      0.34       200

    accuracy                           0.90      2127
   macro avg       0.69      0.62      0.64      2127
weighted avg       0.88      0.90      0.89      2127

              preci